In [58]:
import sqlite3
import pandas as pd

df = pd.read_excel("bhp_final.xlsx")

conn = sqlite3.connect("bhp.db")
df.to_sql("bhp_price", conn, if_exists="replace", index=False)

query = "SELECT * FROM bhp_price;"
pd.read_sql(query, conn)


,ASX Code,Company Name,Date,Close,Volume,Market Capitalisation($m),Shares on Issue,Monthly Returns,Rolling Volatility,Peak,Drawdown,MA12,MA36,Crossover
0,BHP,BHP Group Limited,2005-01-31 00:00:00,13.125,193281753,47023.89,3582772683,-0.149935,0.080163,13.125,0.000000,NaN,NaN,NaN
1,BHP,BHP Group Limited,2005-02-28 00:00:00,15.440,341865362,55318.73,3582772683,0.071032,0.075882,15.440,0.000000,NaN,NaN,NaN
2,BHP,BHP Group Limited,2005-03-31 00:00:00,14.416,332907829,51648.18,3582772683,0.114065,0.077347,15.440,-0.066321,NaN,NaN,NaN
3,BHP,BHP Group Limited,2005-04-29 00:00:00,12.940,394821110,46359.29,3582772683,-0.038347,0.061722,15.440,-0.161917,NaN,NaN,NaN
4,BHP,BHP Group Limited,2005-05-31 00:00:00,13.456,388938234,48209.07,3582772683,-0.081000,0.066457,15.440,-0.128497,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,BHP,BHP Group Limited,2025-04-30 00:00:00,38.190,183378634,193852.14,5075992235,-0.001569,NaN,50.410,-0.242412,41.196667,43.182500,-1.985833
244,BHP,BHP Group Limited,2025-05-30 00:00:00,38.250,175290608,194156.70,5075992235,0.040816,NaN,50.410,-0.241222,40.675000,43.005833,-2.330833
245,BHP,BHP Group Limited,2025-06-30 00:00:00,36.750,200140217,186542.71,5075992235,-0.063694,NaN,50.410,-0.270978,40.180833,42.880833,-2.700000
246,BHP,BHP Group Limited,2025-07-31 00:00:00,39.250,187913083,199232.70,5075992235,-0.091225,NaN,50.410,-0.221385,39.926667,42.896667,-2.970000


---

## Question 1
What was the highest and lowest from each numeric column in the dataset?

---

In [61]:
query = """
SELECT 
    MIN(Close) AS Min_Close, MAX(Close) AS Max_Close, 
    MIN(Volume) AS Min_Volume, MAX(Volume) AS Max_Volume, 
    MIN("Market Capitalisation($m)") AS Min_MarketCap, MAX("Market Capitalisation($m)") AS Max_MarketCap,
    MIN("Shares on Issue") AS Min_Shares, MAX("Shares on Issue") AS Max_Shares
FROM bhp_price;
"""
pd.read_sql(query, conn)


,Min_Close,Max_Close,Min_Volume,Max_Volume,Min_MarketCap,Max_MarketCap,Min_Shares,Max_Shares
0,12.94,50.41,94581510,603912598,43896.11,255515.17,2945851394,5078913175


---

## Question 2

What was the average monthly return for each year?

---

In [66]:
query = """
SELECT 
    strftime('%Y', Date) as YEAR, 
    AVG("Monthly Returns") As Avg_Yearly_Return
FROM bhp_price
GROUP BY Year
ORDER BY Year;
"""

pd.read_sql(query, conn)

,YEAR,Avg_Yearly_Return
0,2005,-0.034725
1,2006,0.001040
2,2007,-0.026526
3,2008,0.023141
4,2009,-0.019472
5,2010,-0.008155
6,2011,0.015682
7,2012,0.001022
8,2013,0.003996
9,2014,0.020041


---

## Question 3

Which year had the worst monthly return, and what was that return?

---

In [80]:
query = """
SELECT 
    strftime('%Y', Date) as YEAR, 
    MIN("Monthly Returns") As Worst_Return
FROM bhp_price
GROUP BY Year
ORDER BY Year
LIMIT 1;
"""

pd.read_sql(query, conn)

,YEAR,Worst_Return
0,2005,-0.149935


---

## Question 4

What was the average trading volume by decade?

---

In [83]:
query = """
SELECT 
    CASE 
        WHEN strftime('%Y', Date) BETWEEN '2005' AND '2009' THEN '2005-2009'
        WHEN strftime('%Y', Date) BETWEEN '2010' AND '2019' THEN '2010-2019'
        ELSE '2020-2025'
    END AS Decade,
    AVG(Volume) AS Avg_Volume
FROM bhp_price
GROUP BY Decade;
"""
pd.read_sql(query, conn)


,Decade,Avg_Volume
0,2005-2009,3.607238e+08
1,2010-2019,1.947788e+08
2,2020-2025,1.726575e+08


---

## Question 5

What were the top 5 best monthly returns, and when did they occur?

---

In [89]:
query = """
SELECT 
    Date,
    "Monthly Returns" AS Return
FROM bhp_price
ORDER BY "Monthly Returns" DESC
LIMIT 5;
"""
pd.read_sql(query, conn)


,Date,Return
0,2008-08-29 00:00:00,0.335453
1,2015-10-30 00:00:00,0.272552
2,2021-08-31 00:00:00,0.212703
3,2021-07-30 00:00:00,0.172761
4,2020-01-31 00:00:00,0.172644
